In [1]:
import subprocess

#### Function for command compatibility with windows cmd and some linux commands

In [2]:
def run_cmd(cmd, **kwargs):
    response = subprocess.Popen(
                    cmd, 
                    stderr=subprocess.PIPE, 
                    stdout=subprocess.PIPE, 
                    **kwargs)

    response.wait()

    stdout, stderr = response.communicate()
    stdout = stdout.decode()
    stderr = stderr.decode()

    returncode = response.returncode

    # print('STDOUT BEGIN \n\n', stdout, '\nSTDOUT END\n\n\n')

    # If had parameter shell=True ignore bug of stderr on windows
    if kwargs.get('shell', False):
        return stdout
        
    else:
        if returncode != 0:
            print(f'Error code:   {returncode}')
            print(stderr)
        else:
            return stdout

## Verifying that devices are available

In [3]:
print(run_cmd('adb devices -l'))

cmd = 'adb devices -l | find "device " /c'
devices_connected = int(run_cmd(cmd, shell=True))
devices_connected

List of devices attached
caa03284               device product:beryllium model:POCOPHONE_F1 device:beryllium transport_id:1




1

In [4]:
def _devices_connected():
    cmd = 'adb devices -l | find "device " /c'
    if int(run_cmd(cmd, shell=True)) > 0:
        return True
    else:
        return False

## Obtaining the device data from the device list of the adb command

In [5]:
def get_devices_data():
    if not _devices_connected():
        return None

    cmd = 'adb devices -l'
    devices_list = run_cmd(cmd).strip().split('\n')[1:]

    for index, device in enumerate(devices_list):

        device = device.strip().split(' ')
        device.reverse()

        if device[4] != 'device':
            continue
        
        device_data = { data.split(':')[0] : data.split(':')[1] for data in device[0:4] }

        device_data['available'] = True if device[4] == 'device' else False
        device_data['serial'] = device[-1]

        devices_list[index] = device_data
    
    return devices_list

In [6]:
devices_list = get_devices_data()
devices_list

[{'transport_id': '1',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': 'caa03284'}]

### Connecting to the device console to get its IP address

In [7]:
serial = devices_list[0]['serial']
serial

'caa03284'

In [8]:
cmd = f'adb -s {serial} shell ip address'
print(run_cmd(cmd))

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: bond0: <BROADCAST,MULTICAST,MASTER> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether fe:85:dc:6c:b6:3e brd ff:ff:ff:ff:ff:ff
3: dummy0: <BROADCAST,NOARP,UP,LOWER_UP> mtu 1500 qdisc noqueue state UNKNOWN group default qlen 1000
    link/ether 3e:61:78:24:f8:4a brd ff:ff:ff:ff:ff:ff
    inet6 fe80::3c61:78ff:fe24:f84a/64 scope link 
       valid_lft forever preferred_lft forever
4: ip_vti0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1
    link/ipip 0.0.0.0 brd 0.0.0.0
5: ip6_vti0@NONE: <NOARP> mtu 1500 qdisc noop state DOWN group default qlen 1
    link/tunnel6 :: brd ::
6: sit0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1


In [9]:
cmd = f'adb -s {serial} shell "ip address | grep -e \'inet .* wlan\'" '
print(run_cmd(cmd))

inet 192.168.1.67/24 brd 192.168.1.255 scope global wlan0



In [10]:
cmd = f'adb -s {serial} shell ' + '"ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '
cmd

'adb -s caa03284 shell "ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '

In [11]:
ip = run_cmd(cmd)
print(ip)

ip = ip.split('/')[0]
print(ip)

192.168.1.67/24

192.168.1.67


In [12]:
def get_device_ip(serial = None):
    if serial is None:
        cmd = 'adb shell "ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '

    else:
        cmd = f'adb -s {serial} shell ' + '"ip address | grep -e \'inet .* wlan\' | awk \'{print $2}\'" '

    return run_cmd(cmd).split('/')[0]

In [13]:
print(get_device_ip())
print(get_device_ip(serial=serial))

192.168.1.67
192.168.1.67


In [14]:
devices_list[0]['ip'] = get_device_ip()
devices_list

[{'transport_id': '1',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': 'caa03284',
  'ip': '192.168.1.67'}]

In [15]:
def tcpip_connect(port = 5555, serial = None, ip = None):
    port = str(port)
    ip = get_device_ip(serial) if ip is None else ip
    
    cmd = f'adb tcpip {port}' if serial is None else f'adb -s {serial} tcpip {port}'
    print(run_cmd(cmd))

    cmd = f'adb connect {ip}:{port}'
    print(run_cmd(cmd))


In [16]:
tcpip_connect()


restarting in TCP mode port: 5555

connected to 192.168.1.67:5555



In [17]:
devices_list = get_devices_data()
devices_list

[{'transport_id': '3',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': 'caa03284'},
 {'transport_id': '2',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': '192.168.1.67:5555'}]

In [18]:
for index,device in enumerate(devices_list):
    devices_list[index]['ip'] = get_device_ip(device['serial'])

devices_list

[{'transport_id': '3',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': 'caa03284',
  'ip': '192.168.1.67'},
 {'transport_id': '2',
  'device': 'beryllium',
  'model': 'POCOPHONE_F1',
  'product': 'beryllium',
  'available': True,
  'serial': '192.168.1.67:5555',
  'ip': '192.168.1.67'}]